# langchain RTFM 

Using a local Ollama and following ...  
https://python.langchain.com/v0.1/docs/get_started/quickstart/

In [28]:
# model select (not part of the guide, guide used llama2

# model_name = "gemma2:9b"
# model_name = "qwen2.5-coder:7b"
# model_name = "llama3.1:latest"  #7b
# model_name = "llama3.2:3b"
# model_name = "llama3.2:1b"
model_name = "llama2:latest"
# model_name = "llama2-uncensored"
# model_name = "tinyllama:latest"

In [29]:
# there's a deprecation warning but this is what the site hase
# mske sure Ollama is up and accessible
from langchain_community.llms import Ollama
llm = Ollama(model=model_name)

## Chaining 
Need the following:  
1. LLM - processes the input and generates an output
2. Prompt Template - define the structure and content of the input query sent to the LLM

In [30]:
# use invoke to query the LLM
llm.invoke("how can langsmith help with testing?")

'\nLangsmith is a language model developed by Google that can assist with testing in several ways:\n\n1. Code Generation: Langsmith can generate code snippets based on natural language inputs, which can be used to test various aspects of an application. For example, you can ask Langsmith to generate a test case for a specific function, and it will produce a ready-to-run test script.\n2. Test Case Generation: Langsmith can generate test cases based on the API documentation or requirements of an application. It can help identify potential test scenarios and generate test cases that cover different paths and edge cases.\n3. Automated Testing: Langsmith can be used to automate testing by generating test scripts that can be executed automatically. This can save time and effort compared to writing and executing tests manually.\n4. Bug Detection: Langsmith can help identify potential bugs in an application by analyzing the code and identifying areas where errors are likely to occur. It can al

In [31]:
# provide a prompt template
input = str("how can langsmith help with testing?")
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

## Simple Chain

In [32]:
# chain them together 
chain = prompt | llm 

In [33]:
# send the new prompt
# this will be the raw output
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a world-class technical documentation writer, I must say that Langsmith is an incredible tool for testing purposes. Here are some ways in which Langsmith can assist you in the testing process:\n\n1. Automated Testing: With Langsmith's advanced AI capabilities, you can automate repetitive tasks such as data entry, text analysis, and content creation. This frees up your time to focus on more critical aspects of testing, such as identifying bugs and edge cases.\n2. Content Generation: Langsmith can generate high-quality content quickly and efficiently, which can be used for testing purposes. For instance, it can create test data sets, test cases, and even automate the creation of documentation for your tests.\n3. Collaboration Tools: Langsmith provides an intuitive collaboration platform that allows you to work with your team more effectively during testing. You can share insights, ideas, and results in real-time, ensuring that everyone is on the same page

## Add to the chain

In [34]:
# add a string parser
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [35]:
# create a new chain with the parser
chain = prompt | llm | output_parser

In [36]:
# send it again
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that LingSMITH is an innovative tool that can greatly assist with testing various aspects of technical documents. Here are some ways in which LangSMITH can help with testing:\n\n1. Grammar and Spelling Checks: LangSMITH's advanced grammar and spelling checks can help identify and correct errors in your technical documentation, ensuring that your content is error-free and polished.\n2. Style Guidance: LangSMITH's style guidance feature can help enforce consistency in formatting, layout, and overall style throughout your technical documents. This consistency is crucial for maintaining a professional image and avoiding confusion.\n3. Terminology Management: With LangSMITH's terminology management feature, you can easily maintain and update your organization's technical terms and definitions. This ensures that your documentation remains accurate and up-to-date.\n4. Compliance Checking: LangSMITH offers compliance checking capab

## Web Retrieval Chain  
1. Get the data
2. Embed in a vector store

In [37]:
# get the data
# requires beautifulsoup4

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [38]:
# OllamaEmbeddings requires llama2
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

In [39]:
# build the index
# requires faiss-cpu

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
# pass the docs object which is the website loaded
documents = text_splitter.split_documents(docs)
# add it to the vector store
vector = FAISS.from_documents(documents, embeddings)

### set up the chain

In [40]:
# without the retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

#### No Retriever

In [41]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by providing a platform for visualizing and analyzing test results.'

In [42]:
#### With the retriever

In [43]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [44]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

According to the provided context, LangSmith can help with testing in several ways:

1. Prototyping: LangSmith provides a platform for rapid experimentation and debugging of LLM applications. Users can quickly test different prompts, models, and retrieval strategies without having to write code.
2. Debugging: When developing new LLM applications, LangSmith tracing is enabled by default. This allows users to easily debug where the model is failing and identify issues.
3. Initial Test Set: LangSmith allows developers to create datasets of inputs and reference outputs, which can be used to run tests on their LLM applications.
4. Comparison View: LangSmith provides a comparison view for test runs, allowing users to track and diagnose regressions in test scores across multiple revisions of the application.
5. Playground: The playground environment allows users to quickly test out different prompts and models without affecting their main application.
6. Beta Testing: LangSmith supports beta 